In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
con = sqlite3.connect('db')

In [3]:
def select(sql):
    return pd.read_sql(sql, con)

In [6]:
t = pd.DataFrame({'dep':['a','a','a','a','a',
                         'b','b','b','b','b'],
                  'emp':['aa','bb','cc','dd','ee',
                         'aa','bb','cc','dd','ee'],
                  'sal':[5,5,3,2,1,
                         5,4,3,2,1]})
t

,dep,emp,sal
0,a,aa,5
1,a,bb,5
2,a,cc,3
3,a,dd,2
4,a,ee,1
5,b,aa,5
6,b,bb,4
7,b,cc,3
8,b,dd,2
9,b,ee,1


In [7]:
t.to_sql('salary',con,index=False,if_exists='replace')

In [14]:
sql ='''select t.*,

 rank() over(partition by t.dep order by t.sal desc) as rnk

 from salary t'''
select(sql)

,dep,emp,sal,rnk
0,a,aa,5,1
1,a,bb,5,1
2,a,cc,3,3
3,a,dd,2,4
4,a,ee,1,5
5,b,aa,5,1
6,b,bb,4,2
7,b,cc,3,3
8,b,dd,2,4
9,b,ee,1,5


In [24]:
sql ='''
WITH
salary_rnk as
(
    select
         t.*,
         rank() over (partition by t.dep order by t.sal desc) as rnk,
         dense_rank() over (partition by t.dep order by t.sal desc) as dsrnk
    from salary t
)


SELECT
    t.*
    FROM  salary_rnk t
    where t.dsrnk <= 3
'''
select(sql)

,dep,emp,sal,rnk,dsrnk
0,a,aa,5,1,1
1,a,bb,5,1,1
2,a,cc,3,3,2
3,a,dd,2,4,3
4,b,aa,5,1,1
5,b,bb,4,2,2
6,b,cc,3,3,3


In [25]:
user1 = pd.DataFrame({'user_id':[1,1,1,1,1],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-01 07:35',
                                       '2021-04-01 08:20','2021-04-01 12:31',
                                       '2021-04-03 07:31'],format='%Y-%m-%d %H:%M')})
user1

,user_id,dt
0,1,2021-04-01 07:31:00
1,1,2021-04-01 07:35:00
2,1,2021-04-01 08:20:00
3,1,2021-04-01 12:31:00
4,1,2021-04-03 07:31:00


In [26]:
user2 = pd.DataFrame({'user_id':[2,2,2,2],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-01 07:35',
                                       '2021-04-01 08:20','2021-04-01 9:10',
                                       ],format='%Y-%m-%d %H:%M')})

In [27]:
user3 = pd.DataFrame({'user_id':[3,3,3],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-02 07:35',
                                       '2021-04-03 08:20'
                                       ],format='%Y-%m-%d %H:%M')})

In [28]:
t = pd.concat([user1,user2,user3])
t

,user_id,dt
0,1,2021-04-01 07:31:00
1,1,2021-04-01 07:35:00
2,1,2021-04-01 08:20:00
3,1,2021-04-01 12:31:00
4,1,2021-04-03 07:31:00
0,2,2021-04-01 07:31:00
1,2,2021-04-01 07:35:00
2,2,2021-04-01 08:20:00
3,2,2021-04-01 09:10:00
0,3,2021-04-01 07:31:00


In [29]:
t.to_sql('client_log',con,index=False,if_exists='replace')

In [52]:
sql ='''
with
new_session as (
select t.*,
 lag(t.dt) over (partition by t.user_id order by t.dt) as prev_dt,
 (julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60 as dt_diff,
 case when (julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60 >= 3600
 then 1 else 0 end as new_session
 from client_log t),

 client_sessions as (
 select t.*,
  sum(t.new_session) over (partition by t.user_id order by t.dt) as session_id
  from new_session t),

 client_sessions_agg as(
  select t.user_id, t.session_id, count(1)
   from client_sessions t
   group by t.user_id, t.session_id)

select count(1) from client_sessions_agg t
 '''
select(sql)

,count(1)
0,7


In [55]:
t = pd.DataFrame({'user_id':[1,1,1,1,1,1,
                             2,2,2,2,2],
                  'dt':[1,2,3,4,5,6,
                        1,2,3,4,5],
                  'revenue':[1,2,3,4,5,6,
                             3,4,5,6,7]})
t

,user_id,dt,revenue
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5
5,1,6,6
6,2,1,3
7,2,2,4
8,2,3,5
9,2,4,6


In [57]:
t.to_sql('revenue',con,index=False,if_exists='replace')

In [61]:
sql ='''select t.*,
 avg(t.revenue) over (partition by t.user_id order by t.dt
 rows between 2 preceding and current row) as moving_avg
 from revenue t'''
select(sql)

,user_id,dt,revenue,moving_avg
0,1,1,1,1.0
1,1,2,2,1.5
2,1,3,3,2.0
3,1,4,4,3.0
4,1,5,5,4.0
5,1,6,6,5.0
6,2,1,3,3.0
7,2,2,4,3.5
8,2,3,5,4.0
9,2,4,6,5.0
